In [1]:
import logging
import os 
%pwd

'd:\\My Workspace\\proj_file\\End-to-End-Image-Classification-Project\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\My Workspace\\proj_file\\End-to-End-Image-Classification-Project'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL : str
    local_data_file: Path
    unzip_dir: Path
    


In [4]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        logging.info("paths of the yaml files are assinged")
        
        create_directories([self.config.artifacts_root])
        
        logging.info("aritifacts folder created")
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        logging.info("data ingestion folder  created")
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        logging.info("successfully returned the DataIngestionConfig")
        
        logging.info("ConfigurationManager for data ingestion done")
        return data_ingestion_config


In [5]:

import os 
import urllib.request as request
from zipfile import ZipFile
import logging
from src.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logging.info(f"{filename} download! with following info: \n{headers}")
        else:
            logging.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("cat" in f or "dog" in f)]
    
    
    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)
            logging.info("extracted all the folders and files from the zip file")
        

In [7]:
from src.exception import CustomException
import sys
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise CustomException(e,sys)

logging.info("data ingestion successfully completed")

[ 19-07-2024 04:19:59 PM ] root , line : 34 - INFO - yaml file: config\config.yaml loaded successfully
[ 19-07-2024 04:19:59 PM ] root , line : 34 - INFO - yaml file: params.yaml loaded successfully
[ 19-07-2024 04:19:59 PM ] root , line : 34 - INFO - yaml file: schema.yaml loaded successfully
[ 19-07-2024 04:19:59 PM ] root , line : 15 - INFO - paths of the yaml files are assinged
[ 19-07-2024 04:19:59 PM ] root , line : 54 - INFO - created directory at: artifacts
[ 19-07-2024 04:19:59 PM ] root , line : 19 - INFO - aritifacts folder created
[ 19-07-2024 04:19:59 PM ] root , line : 54 - INFO - created directory at: artifacts/data_ingestion
[ 19-07-2024 04:19:59 PM ] root , line : 27 - INFO - data ingestion folder  created
[ 19-07-2024 04:19:59 PM ] root , line : 36 - INFO - successfully returned the DataIngestionConfig
[ 19-07-2024 04:19:59 PM ] root , line : 38 - INFO - ConfigurationManager for data ingestion done
[ 19-07-2024 04:20:02 PM ] root , line : 19 - INFO - artifacts/data_in